Model exploration
===

In [1]:
from transformers import BertModel, DistilBertModel, AutoTokenizer

# 1. Load the Tokenizer (shared or compatible for both)
# It's common to use the teacher's tokenizer or a tokenizer compatible with both.
# For DistilBERT, the tokenizer is often the same as BERT's.
tokenizer_name = 'bert-base-uncased' # Or 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# 2. Load the Teacher Model (BERT-base)
teacher_model_name = 'bert-base-uncased'
teacher_model = BertModel.from_pretrained(teacher_model_name)
teacher_model.eval() # Set to evaluation mode, as we're not training it here.

# 3. Load the Student Model (DistilBERT-base)
student_model_name = 'distilbert-base-uncased' # This is the model the paper's authors trained and released
student_model = DistilBertModel.from_pretrained(student_model_name)
student_model.eval() # Also set to evaluation mode for now.

# You can print the models to see their architecture (it will be quite long)
# print("Teacher Model (BERT-base):")
# print(teacher_model)
# print("\nStudent Model (DistilBERT-base):")
# print(student_model)

print(f"Successfully loaded tokenizer: {tokenizer_name}")
print(f"Successfully loaded teacher model: {teacher_model_name}")
print(f"Successfully loaded student model: {student_model_name}")

# Let's check the number of parameters to see the size difference
teacher_params = sum(p.numel() for p in teacher_model.parameters())
student_params = sum(p.numel() for p in student_model.parameters())

print(f"\nTeacher model parameters: {teacher_params:,}")
print(f"Student model parameters: {student_params:,}")
print(f"Student model is roughly {student_params/teacher_params:.2%} the size of the teacher model.")

Successfully loaded tokenizer: bert-base-uncased
Successfully loaded teacher model: bert-base-uncased
Successfully loaded student model: distilbert-base-uncased

Teacher model parameters: 109,482,240
Student model parameters: 66,362,880
Student model is roughly 60.62% the size of the teacher model.


In [2]:
from datasets import load_dataset

# Load the SST-2 dataset from Hugging Face Datasets
sst2_dataset = load_dataset('glue', 'sst2')

# The dataset object contains train, validation (which we'll use as development), and test splits
print(sst2_dataset)

# Use the tokenizer we loaded earlier

# Define a function to tokenize the text data with padding and truncation
max_length = 128
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding='max_length', max_length=max_length)

# Apply the tokenization to all splits of the dataset
tokenized_sst2 = sst2_dataset.map(tokenize_function, batched=True)

# The 'label' column is already numerical, so we don't need to process it further for SST-2.
# We will, however, remove the original 'sentence' column as the model will use the tokenized inputs.
tokenized_sst2 = tokenized_sst2.remove_columns(["sentence", "idx"])

# Rename the 'label' column to 'labels' as that's the expected name by PyTorch models
tokenized_sst2 = tokenized_sst2.rename_column("label", "labels")

print(tokenized_sst2)
print(tokenized_sst2["train"][:1])

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})
{'labels': [0], 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Training
===

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DistilBertForSequenceClassification
import evaluate
import numpy as np

repository_name = "LukasXperiaZ/distilBERT_SST-2"

# 1. Define Training Arguments
output_dir = './distilbert-sst2' # Directory to save the trained model and logs
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=512,    # 512 good for 24 GB VRAM
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch", # Save the model at the end of each epoch
    load_best_model_at_end=True, # Load the best model based on evaluation metric
    metric_for_best_model="accuracy",
    report_to="none", # Or "wandb" if you want to use Weights & Biases for logging
    hub_model_id=repository_name
)

# 2. Load the Pre-trained Model for Sequence Classification
num_labels = len(sst2_dataset['train'].features['label'].names) # Get the number of unique labels (0 and 1 for SST-2)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# 3. Load the Evaluation Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 4. Create a Trainer Instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst2["train"],
    eval_dataset=tokenized_sst2["validation"], # Using the 'validation' split as the development set
    compute_metrics=compute_metrics,
)

print("Trainer and model are ready for training!")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainer and model are ready for training!


In [4]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.259829,0.900229
2,No log,0.263222,0.897936
3,No log,0.270581,0.902523
4,0.194700,0.287648,0.902523
5,0.194700,0.292909,0.900229


TrainOutput(global_step=660, training_loss=0.17371627056237424, metrics={'train_runtime': 340.517, 'train_samples_per_second': 988.923, 'train_steps_per_second': 1.938, 'total_flos': 1.115193354010368e+16, 'train_loss': 0.17371627056237424, 'epoch': 5.0})

In [ ]:
# Upload the model to huggingface
trainer.push_to_hub(commit_message="Add best DistilBERT model trained on SST-2")
tokenizer.push_to_hub(
    repo_id=repository_name,
    commit_message="Upload BERT tokenizer for DistilBERT model",
    blocking=True,  # Wait until the push is complete
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LukasXperiaZ/distilBERT_SST-2/commit/59bcae90208d6f7755abede21d4c0be1771e99b8', commit_message='Add best DistilBERT model trained on SST-2', commit_description='', oid='59bcae90208d6f7755abede21d4c0be1771e99b8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/LukasXperiaZ/distilBERT_SST-2', endpoint='https://huggingface.co', repo_type='model', repo_id='LukasXperiaZ/distilBERT_SST-2'), pr_revision=None, pr_num=None)

Validation
===
Execute from here on if you just want to evaluate

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
import evaluate
import torch
import numpy as np

# 1. Specify your repository name on Hugging Face Hub
repository_name = 'LukasXperiaZ/distilBERT_SST-2'

# 2. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(repository_name)

# 3. Load the model from Hugging Face Hub
model = AutoModelForSequenceClassification.from_pretrained(repository_name)

# 4. Load the SST-2 validation dataset and tokenize it (if not already done)
sst2_validation_dataset = load_dataset('glue', 'sst2', split='validation')

def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding='max_length', max_length=128)

tokenized_sst2_validation = sst2_validation_dataset.map(tokenize_function, batched=True)
tokenized_sst2_validation = tokenized_sst2_validation.remove_columns(["sentence", "idx"])
tokenized_sst2_validation = tokenized_sst2_validation.rename_column("label", "labels")
tokenized_sst2_validation.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 5. Create a DataLoader for the validation set
eval_dataloader = DataLoader(tokenized_sst2_validation, batch_size=32) # Use the same eval batch size

# 6. Define the evaluation metric
metric = evaluate.load("accuracy")

# 7. Move the model to the appropriate device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()

# 8. Evaluation loop
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

# 9. Print the evaluation results
eval_metric = metric.compute()
print(f"Validation set results (loaded from Hugging Face Hub): {eval_metric}")

Validation set results (loaded from Hugging Face Hub): {'accuracy': 0.9025229357798165}
